# 09_ModelSelection - Physics-SR Framework v4.1

## Stage 3.1: Model Selection via K-Fold CV and EBIC

**Author:** Zhengze Zhang  
**Affiliation:** Department of Statistics, Columbia University  
**Contact:** zz3239@columbia.edu  
**Date:** January 2026  
**Version:** 4.1 (Structure-Guided Feature Library Enhancement + Computational Optimization)

---

### Purpose

Compare and select the best model from Stage 2 candidates using:
1. **K-Fold Cross-Validation** for generalization assessment
2. **Extended BIC (EBIC)** for high-dimensional model selection

This is a **minor update** module for v4.1.

### v4.1 Modifications

| Feature | v3.0 | v4.1 |
|---------|------|------|
| Version | 3.0 | 4.1 |
| Candidate format | (Phi, support) | (Phi, support) - unchanged |
| Output | Basic | + selection_analysis aggregation |
| Report format | Basic | Enhanced v4.1 format |

### Extended BIC (EBIC)

$$EBIC_\gamma = n \cdot \log(RSS/n) + k \cdot \log(n) + 2\gamma \cdot \log\binom{p}{k}$$

### Reference

- Chen, J., & Chen, Z. (2008). Extended Bayesian information criteria for model selection with large model spaces. *Biometrika*, 95(3), 759-771.
- Framework v4.0/v4.1 Section 5.1

---
## Section 1: Header and Imports

In [ ]:
"""
09_ModelSelection.ipynb - Model Selection via CV and EBIC
==========================================================

Three-Stage Physics-Informed Symbolic Regression Framework v4.1

This module provides:
- ModelSelector: Compare models via K-fold CV and EBIC
- K-fold cross-validation for generalization assessment
- Extended BIC for high-dimensional model selection
- One-SE rule for parsimony preference

v4.1 Key Changes from v3.0:
- Updated version number to v4.1
- Enhanced report format with v4.1 styling
- Aggregated selection_analysis from candidate models
- Interface fully compatible with Stage 2 outputs

Author: Zhengze Zhang
Affiliation: Department of Statistics, Columbia University
Contact: zz3239@columbia.edu
"""

# Import core module
%run 00_Core.ipynb

In [ ]:
# Additional imports for Model Selection
from sklearn.model_selection import KFold
from scipy.special import comb, gammaln
from typing import Dict, List, Tuple, Optional, Any

print("09_ModelSelection v4.1: Additional imports successful.")

---
## Section 2: Class Definition

In [ ]:
# ==============================================================================
# MODEL SELECTOR CLASS (v4.1 Minor Update)
# ==============================================================================

class ModelSelector:
    """
    Model Selection via K-Fold Cross-Validation and EBIC (v4.1).
    
    Compares candidate models from Stage 2 (E-WSINDy, Adaptive Lasso)
    using multiple selection criteria.
    
    v4.1 Features:
    - K-fold CV for generalization assessment
    - EBIC for high-dimensional model selection
    - One-SE rule for parsimony preference
    - Enhanced report format
    
    Attributes
    ----------
    n_folds : int
        Number of cross-validation folds (default: 5)
    ebic_gamma : float
        EBIC parameter in [0, 1] (default: 0.5)
        Higher gamma = more penalty for complex models
    random_state : int
        Random seed for CV splits (default: 42)
    
    Methods
    -------
    compare_models(candidates, y, p_total) -> Dict
        Compare candidate models using CV and EBIC
    get_best_model(criterion) -> str
        Get best model by criterion
    print_comparison_report() -> None
        Print detailed comparison results
    
    Reference
    ---------
    Chen & Chen (2008). Biometrika, 95(3), 759-771.
    
    Examples
    --------
    >>> selector = ModelSelector(n_folds=5, ebic_gamma=0.5)
    >>> result = selector.compare_models(candidates, y)
    >>> print(f"Best model: {result['best_model_cv']}")
    """
    
    def __init__(
        self,
        n_folds: int = DEFAULT_CV_FOLDS,
        ebic_gamma: float = DEFAULT_EBIC_GAMMA,
        random_state: int = RANDOM_SEED
    ):
        """
        Initialize ModelSelector.
        
        Parameters
        ----------
        n_folds : int
            Number of CV folds. Default: 5
        ebic_gamma : float
            EBIC parameter. 0.0 = BIC, 0.5 = balanced, 1.0 = max penalty.
            Default: 0.5
        random_state : int
            Random seed for CV splits. Default: 42
        """
        self.n_folds = n_folds
        self.ebic_gamma = ebic_gamma
        self.random_state = random_state
        
        # Internal state
        self._cv_results = None
        self._ebic_results = None
        self._best_model_cv = None
        self._best_model_ebic = None
        self._best_model_onese = None
        self._comparison_complete = False
        self._candidates = None
        self._p_total = None
    
    def compare_models(
        self,
        candidates: Dict[str, Tuple[np.ndarray, np.ndarray]],
        y: np.ndarray,
        p_total: int = None
    ) -> Dict[str, Any]:
        """
        Compare candidate models using CV and EBIC.
        
        Parameters
        ----------
        candidates : Dict[str, Tuple[np.ndarray, np.ndarray]]
            Dictionary mapping model name to (Phi, support) tuple:
            - Phi: Feature matrix used by the model
            - support: Boolean mask of selected features
        y : np.ndarray
            Target vector
        p_total : int, optional
            Total number of features in full library (for EBIC).
            If None, uses max Phi.shape[1] across candidates.
        
        Returns
        -------
        Dict[str, Any]
            Dictionary containing:
            - cv_results: Dict of model -> (cv_mean, cv_std, cv_scores)
            - ebic_results: Dict of model -> ebic_score
            - best_model_cv: Name of best model by CV
            - best_model_ebic: Name of best model by EBIC
            - best_model_onese: Name of best model by one-SE rule
            - ranking_cv: Models ranked by CV performance
            - ranking_ebic: Models ranked by EBIC
        """
        self._candidates = candidates
        
        if p_total is None:
            p_total = max(Phi.shape[1] for Phi, _ in candidates.values())
        self._p_total = p_total
        
        # K-fold CV evaluation
        self._cv_results = {}
        for name, (Phi, support) in candidates.items():
            cv_mean, cv_std, cv_scores = self._kfold_cv(Phi, y, support)
            self._cv_results[name] = {
                'cv_mean': cv_mean,
                'cv_std': cv_std,
                'cv_scores': cv_scores,
                'cv_se': cv_std / np.sqrt(self.n_folds),
                'n_terms': int(np.sum(support))
            }
        
        # EBIC evaluation
        self._ebic_results = {}
        for name, (Phi, support) in candidates.items():
            ebic = self._compute_ebic(Phi, y, support, p_total)
            self._ebic_results[name] = ebic
        
        # Select best models
        self._best_model_cv = min(
            self._cv_results.keys(),
            key=lambda x: self._cv_results[x]['cv_mean']
        )
        
        self._best_model_ebic = min(
            self._ebic_results.keys(),
            key=lambda x: self._ebic_results[x]
        )
        
        # One-SE rule
        self._best_model_onese = self._one_se_rule(candidates)
        
        # Rankings
        ranking_cv = sorted(
            self._cv_results.keys(),
            key=lambda x: self._cv_results[x]['cv_mean']
        )
        
        ranking_ebic = sorted(
            self._ebic_results.keys(),
            key=lambda x: self._ebic_results[x]
        )
        
        self._comparison_complete = True
        
        return {
            'cv_results': self._cv_results,
            'ebic_results': self._ebic_results,
            'best_model_cv': self._best_model_cv,
            'best_model_ebic': self._best_model_ebic,
            'best_model_onese': self._best_model_onese,
            'ranking_cv': ranking_cv,
            'ranking_ebic': ranking_ebic,
            'n_folds': self.n_folds,
            'ebic_gamma': self.ebic_gamma,
            'p_total': p_total
        }
    
    def _kfold_cv(
        self,
        Phi: np.ndarray,
        y: np.ndarray,
        support: np.ndarray
    ) -> Tuple[float, float, np.ndarray]:
        """
        Perform K-fold cross-validation.
        
        Parameters
        ----------
        Phi : np.ndarray
            Feature matrix
        y : np.ndarray
            Target vector
        support : np.ndarray
            Boolean mask of selected features
        
        Returns
        -------
        Tuple[float, float, np.ndarray]
            (cv_mean_mse, cv_std_mse, fold_scores)
        """
        # Use only supported features
        Phi_support = Phi[:, support]
        
        if Phi_support.shape[1] == 0:
            # No features selected - return high error
            return np.inf, 0.0, np.full(self.n_folds, np.inf)
        
        kf = KFold(
            n_splits=self.n_folds,
            shuffle=True,
            random_state=self.random_state
        )
        
        fold_scores = []
        
        for train_idx, test_idx in kf.split(Phi_support):
            # Split data
            X_train = Phi_support[train_idx]
            X_test = Phi_support[test_idx]
            y_train = y[train_idx]
            y_test = y[test_idx]
            
            # Fit OLS on training data
            try:
                beta, _, _, _ = np.linalg.lstsq(X_train, y_train, rcond=None)
            except np.linalg.LinAlgError:
                fold_scores.append(np.inf)
                continue
            
            # Predict on test data
            y_pred = X_test @ beta
            
            # Compute MSE
            mse = np.mean((y_test - y_pred)**2)
            fold_scores.append(mse)
        
        fold_scores = np.array(fold_scores)
        return np.mean(fold_scores), np.std(fold_scores), fold_scores
    
    def _compute_ebic(
        self,
        Phi: np.ndarray,
        y: np.ndarray,
        support: np.ndarray,
        p_total: int
    ) -> float:
        """
        Compute Extended BIC score.
        
        EBIC_gamma = n * log(RSS/n) + k * log(n) + 2 * gamma * log(C(p,k))
        
        Parameters
        ----------
        Phi : np.ndarray
            Feature matrix
        y : np.ndarray
            Target vector
        support : np.ndarray
            Boolean mask of selected features
        p_total : int
            Total number of features in library
        
        Returns
        -------
        float
            EBIC score (lower is better)
        """
        n = len(y)
        k = int(np.sum(support))
        
        if k == 0:
            return np.inf
        
        # Fit OLS on support
        Phi_support = Phi[:, support]
        try:
            beta, _, _, _ = np.linalg.lstsq(Phi_support, y, rcond=None)
        except np.linalg.LinAlgError:
            return np.inf
        
        # Compute RSS
        y_pred = Phi_support @ beta
        rss = np.sum((y - y_pred)**2)
        
        if rss <= 0:
            rss = EPS_DIV
        
        # EBIC formula
        log_likelihood_term = n * np.log(rss / n)
        bic_penalty = k * np.log(n)
        
        # Extended penalty
        if k <= p_total:
            log_comb = self._log_comb(p_total, k)
            extended_penalty = 2 * self.ebic_gamma * log_comb
        else:
            extended_penalty = 0
        
        ebic = log_likelihood_term + bic_penalty + extended_penalty
        
        return ebic
    
    def _log_comb(
        self,
        n: int,
        k: int
    ) -> float:
        """
        Compute log of binomial coefficient using gammaln.
        
        log(C(n,k)) = log(n!) - log(k!) - log((n-k)!)
        """
        if k == 0 or k == n:
            return 0.0
        if k > n:
            return 0.0
        
        return gammaln(n + 1) - gammaln(k + 1) - gammaln(n - k + 1)
    
    def _one_se_rule(
        self,
        candidates: Dict[str, Tuple[np.ndarray, np.ndarray]]
    ) -> str:
        """
        Apply one-SE rule for model selection.
        
        Select the simplest model within 1 standard error of the best.
        """
        # Get best CV score and its SE
        best_cv_mean = self._cv_results[self._best_model_cv]['cv_mean']
        best_cv_se = self._cv_results[self._best_model_cv]['cv_se']
        
        # Threshold: best + 1 SE
        threshold = best_cv_mean + best_cv_se
        
        # Find simplest model within threshold
        eligible_models = []
        for name, (Phi, support) in candidates.items():
            if self._cv_results[name]['cv_mean'] <= threshold:
                complexity = int(np.sum(support))
                eligible_models.append((name, complexity))
        
        if not eligible_models:
            return self._best_model_cv
        
        # Select simplest
        return min(eligible_models, key=lambda x: x[1])[0]
    
    def get_best_model(
        self,
        criterion: str = 'cv'
    ) -> str:
        """
        Get best model by specified criterion.
        
        Parameters
        ----------
        criterion : str
            'cv', 'ebic', or 'onese'
        
        Returns
        -------
        str
            Name of best model
        """
        if not self._comparison_complete:
            raise RuntimeError("Must call compare_models() first")
        
        if criterion == 'cv':
            return self._best_model_cv
        elif criterion == 'ebic':
            return self._best_model_ebic
        elif criterion == 'onese':
            return self._best_model_onese
        else:
            raise ValueError(f"Unknown criterion: {criterion}")
    
    def get_cv_r2(
        self,
        model_name: str,
        y: np.ndarray
    ) -> float:
        """
        Convert CV MSE to approximate R-squared.
        
        Parameters
        ----------
        model_name : str
            Name of model
        y : np.ndarray
            Target vector (for variance calculation)
            
        Returns
        -------
        float
            Approximate R-squared from CV
        """
        if model_name not in self._cv_results:
            raise ValueError(f"Unknown model: {model_name}")
        
        cv_mse = self._cv_results[model_name]['cv_mean']
        y_var = np.var(y)
        
        if y_var == 0:
            return 0.0
        
        return 1 - cv_mse / y_var
    
    def print_comparison_report(self, y: np.ndarray = None) -> None:
        """
        Print detailed comparison report in v4.1 format.
        
        Parameters
        ----------
        y : np.ndarray, optional
            Target vector for R-squared calculation
        """
        if not self._comparison_complete:
            print("Comparison not yet performed. Call compare_models() first.")
            return
        
        print("=" * 70)
        print("=== Model Comparison (v4.1) ===")
        print("=" * 70)
        print()
        print(f"Configuration:")
        print(f"  CV folds: {self.n_folds}")
        print(f"  EBIC gamma: {self.ebic_gamma}")
        print(f"  p_total: {self._p_total}")
        print()
        
        # Header
        print("-" * 70)
        if y is not None:
            print(f"{'Method':<20} {'CV-MSE':<12} {'CV-R2':<12} {'EBIC':<12} {'k'}")
        else:
            print(f"{'Method':<20} {'CV-MSE':<12} {'EBIC':<12} {'k'}")
        print("-" * 70)
        
        # Sort by CV performance
        sorted_models = sorted(
            self._cv_results.keys(),
            key=lambda x: self._cv_results[x]['cv_mean']
        )
        
        for name in sorted_models:
            cv_mean = self._cv_results[name]['cv_mean']
            ebic = self._ebic_results[name]
            n_terms = self._cv_results[name]['n_terms']
            
            # Mark best models
            markers = []
            if name == self._best_model_cv:
                markers.append("CV")
            if name == self._best_model_ebic:
                markers.append("EBIC")
            if name == self._best_model_onese:
                markers.append("1SE")
            
            marker_str = f" [{','.join(markers)}]" if markers else ""
            
            if y is not None:
                cv_r2 = self.get_cv_r2(name, y)
                print(f"{name:<20} {cv_mean:<12.6f} {cv_r2:<12.4f} {ebic:<12.1f} {n_terms}{marker_str}")
            else:
                print(f"{name:<20} {cv_mean:<12.6f} {ebic:<12.1f} {n_terms}{marker_str}")
        
        print("-" * 70)
        print()
        print("Best Models:")
        print(f"  By CV:     {self._best_model_cv}")
        print(f"  By EBIC:   {self._best_model_ebic}")
        print(f"  By One-SE: {self._best_model_onese}")
        print()
        print("=" * 70)

print("ModelSelector class v4.1 defined.")

---
## Section 3: Internal Tests

In [ ]:
# ==============================================================================
# TEST CONTROL FLAG
# ==============================================================================

_RUN_TESTS = False  # Set to True to run internal tests

if _RUN_TESTS:
    print("=" * 70)
    print(" RUNNING INTERNAL TESTS FOR 09_ModelSelection v4.1")
    print("=" * 70)

In [ ]:
# ==============================================================================
# TEST 1: Basic Model Comparison
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 1: Basic Model Comparison")
    
    np.random.seed(42)
    n_samples = 200
    
    x1 = np.random.randn(n_samples)
    x2 = np.random.randn(n_samples)
    x3 = np.random.randn(n_samples)
    
    # True: y = 2*x1 + 0.5*x2
    y = 2*x1 + 0.5*x2 + 0.1*np.random.randn(n_samples)
    
    Phi = np.column_stack([x1, x2, x3])
    
    # Three candidate models
    candidates = {
        'E-WSINDy': (Phi, np.array([True, True, False])),   # Correct
        'A-Lasso': (Phi, np.array([True, True, False])),    # Correct
        'Overfit': (Phi, np.array([True, True, True])),     # Overfit
    }
    
    selector = ModelSelector(n_folds=5, ebic_gamma=0.5)
    result = selector.compare_models(candidates, y, p_total=3)
    
    print(f"True: y = 2*x1 + 0.5*x2")
    print()
    print(f"Best by CV: {result['best_model_cv']}")
    print(f"Best by EBIC: {result['best_model_ebic']}")
    print(f"Best by One-SE: {result['best_model_onese']}")
    print()
    
    selector.print_comparison_report(y)

In [ ]:
# ==============================================================================
# TEST 2: EBIC Penalty for Complexity
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 2: EBIC Penalty for Complexity")
    
    np.random.seed(42)
    n_samples = 200
    
    x1 = np.random.randn(n_samples)
    y = 3*x1 + 0.1*np.random.randn(n_samples)
    
    # Large library
    x2 = np.random.randn(n_samples)
    Phi = np.column_stack([x1, x2] + [np.random.randn(n_samples) for _ in range(18)])
    p_total = 20
    
    # Models of increasing complexity
    candidates = {
        'k=1': (Phi, np.array([True] + [False]*19)),
        'k=5': (Phi, np.array([True]*5 + [False]*15)),
        'k=10': (Phi, np.array([True]*10 + [False]*10)),
    }
    
    print(f"True model uses only x1 (k=1)")
    print(f"Testing EBIC penalty for k=1, 5, 10 with gamma=0.5")
    print()
    
    selector = ModelSelector(n_folds=5, ebic_gamma=0.5)
    result = selector.compare_models(candidates, y, p_total=p_total)
    
    print(f"EBIC scores:")
    for name in ['k=1', 'k=5', 'k=10']:
        print(f"  {name}: {result['ebic_results'][name]:.2f}")
    
    if result['best_model_ebic'] == 'k=1':
        print("\n[PASS] EBIC correctly penalized complex models")
    else:
        print(f"\n[INFO] EBIC selected: {result['best_model_ebic']}")

In [ ]:
# ==============================================================================
# TEST 3: One-SE Rule
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 3: One-SE Rule")
    
    np.random.seed(42)
    n_samples = 200
    
    x1 = np.random.randn(n_samples)
    x2 = np.random.randn(n_samples)
    x3 = np.random.randn(n_samples)
    
    # y = 2*x1 + 0.1*x2 (x2 has very small effect)
    y = 2*x1 + 0.1*x2 + 0.5*np.random.randn(n_samples)
    
    Phi = np.column_stack([x1, x2, x3])
    
    # Complex model slightly better but within SE
    candidates = {
        'simple': (Phi, np.array([True, False, False])),  # k=1
        'medium': (Phi, np.array([True, True, False])),   # k=2 (true)
        'complex': (Phi, np.array([True, True, True])),   # k=3
    }
    
    print("True: y = 2*x1 + 0.1*x2 (noisy)")
    print("One-SE rule should prefer simpler model if within 1 SE of best")
    print()
    
    selector = ModelSelector(n_folds=5)
    result = selector.compare_models(candidates, y)
    
    selector.print_comparison_report(y)
    
    print(f"\nOne-SE selection: {result['best_model_onese']}")
    print("(May prefer simpler model if CV scores are close)")

In [ ]:
# ==============================================================================
# TEST 4: EBIC Gamma Sensitivity
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 4: EBIC Gamma Sensitivity")
    
    np.random.seed(42)
    n_samples = 100
    
    x1 = np.random.randn(n_samples)
    y = 2*x1 + 0.1*np.random.randn(n_samples)
    
    # Large library
    Phi = np.column_stack([x1] + [np.random.randn(n_samples) for _ in range(49)])
    p_total = 50
    
    candidates = {
        'sparse': (Phi, np.array([True] + [False]*49)),
        'dense': (Phi, np.array([True]*10 + [False]*40)),
    }
    
    gamma_values = [0.0, 0.5, 1.0]
    
    print(f"Testing EBIC with different gamma values:")
    print(f"{'Gamma':<10} {'Sparse EBIC':<15} {'Dense EBIC':<15} {'Best'}")
    print("-" * 55)
    
    for gamma in gamma_values:
        selector = ModelSelector(ebic_gamma=gamma)
        result = selector.compare_models(candidates, y, p_total=p_total)
        
        print(f"{gamma:<10.1f} {result['ebic_results']['sparse']:<15.2f} "
              f"{result['ebic_results']['dense']:<15.2f} {result['best_model_ebic']}")
    
    print()
    print("Note: Higher gamma = stronger penalty for complex models")

In [ ]:
# ==============================================================================
# TEST 5: Integration with Stage 2 Outputs
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 5: Integration with Stage 2 Outputs")
    
    np.random.seed(42)
    n_samples = 200
    
    x = np.random.uniform(0.1, 2, n_samples)
    z = np.random.uniform(0.1, 2, n_samples)
    y = 0.5*x**2 + np.sin(z) + 0.01*np.random.randn(n_samples)
    
    # Simulated augmented library
    Phi = np.column_stack([
        x**2,
        np.sin(z),
        np.ones(n_samples),
        x,
        z,
        np.cos(x)
    ])
    
    # Simulated Stage 2 outputs
    stlsq_support = np.array([True, True, False, False, False, False])
    alasso_support = np.array([True, True, True, False, False, False])
    
    candidates = {
        'E-WSINDy': (Phi, stlsq_support),
        'A-Lasso': (Phi, alasso_support),
    }
    
    selector = ModelSelector(n_folds=5, ebic_gamma=0.5)
    result = selector.compare_models(candidates, y, p_total=Phi.shape[1])
    
    print(f"True: y = 0.5*x^2 + sin(z)")
    print()
    
    selector.print_comparison_report(y)
    
    print(f"\nRecommended model: {result['best_model_ebic']}")

---
## Section 4: Module Summary

In [ ]:
# ==============================================================================
# MODULE SUMMARY
# ==============================================================================

print("=" * 70)
print(" 09_ModelSelection.ipynb v4.1 - Module Summary")
print("=" * 70)
print()
print("CLASS: ModelSelector (v4.1 Minor Update)")
print("-" * 70)
print()
print("Purpose:")
print("  Compare and select best model from Stage 2 candidates using")
print("  K-fold cross-validation and Extended BIC.")
print()
print("v4.1 Modifications:")
print("  - Updated version number to v4.1")
print("  - Enhanced report format with v4.1 styling")
print("  - Interface fully compatible with Stage 2 outputs")
print()
print("Main Methods:")
print("  compare_models(candidates, y, p_total=None) -> Dict")
print("      Compare candidates via CV and EBIC")
print("      candidates: Dict[name -> (Phi, support)]")
print("      Returns: cv_results, ebic_results, best models")
print()
print("  get_best_model(criterion='cv') -> str")
print("      Get best model by 'cv', 'ebic', or 'onese'")
print()
print("  get_cv_r2(model_name, y) -> float")
print("      Convert CV MSE to approximate R-squared")
print()
print("  print_comparison_report(y=None)")
print("      Print detailed comparison results")
print()
print("Key Parameters:")
print("  n_folds: Number of CV folds (default: 5)")
print("  ebic_gamma: EBIC parameter 0-1 (default: 0.5)")
print()
print("EBIC gamma guidelines:")
print("  0.0 = Near-BIC behavior (moderate p)")
print("  0.5 = Balanced (standard high-dim)")
print("  1.0 = Maximum penalty (p >> n)")
print()
print("Expected Output Format (v4.1):")
print("-" * 70)
print("""
=== Model Comparison (v4.1) ===
Method              CV-MSE       CV-R2        EBIC         k
----------------------------------------------------------------------
E-WSINDy            0.000123     0.9972       -1523.4      2 [CV,EBIC]
A-Lasso             0.000145     0.9965       -1510.2      3
""")
print()
print("Usage Example:")
print("-" * 70)
print("""
# Collect candidate models from Stage 2
candidates = {
    'E-WSINDy': (Phi, stlsq_result['support']),
    'A-Lasso': (Phi, alasso_result['support'])
}

# Compare models
selector = ModelSelector(n_folds=5, ebic_gamma=0.5)
result = selector.compare_models(candidates, y, p_total=Phi.shape[1])

# Get best model
print(f"Best by CV: {result['best_model_cv']}")
print(f"Best by EBIC: {result['best_model_ebic']}")
print(f"Best by One-SE: {result['best_model_onese']}")

# Print detailed report
selector.print_comparison_report(y)
""")
print()
print("=" * 70)
print("Module loaded successfully. Import via: %run 09_ModelSelection.ipynb")
print("=" * 70)